In [63]:
import pandas as pd
from sqlalchemy import create_engine

## connect to Database

In [64]:
engine = create_engine("sqlite:///data/MTA_2019.db")

## Get data from MTA website

In [ ]:
# Source: http://web.mta.info/developers/turnstile.html
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [190928,190921,190914,190907,191005,191012,191019,191026,191102,191109,191116
             ,191123,191130]
turnstiles_df = get_data(week_nums)

In [ ]:
#remove space from column name
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns] 

#trasform date tp datetime datatype
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")

In [ ]:
#see if there is duplicates
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          ascending=False).head(20)

In [ ]:
# delete rows if desc is not regular
turnstiles_df=turnstiles_df.loc[turnstiles_df['DESC'] == "REGULAR"]

In [ ]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", 
           "DATE_TIME","LINENAME"])[["ENTRIES"]].count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(10)

In [ ]:
import numpy as np

station_df =turnstiles_df.groupby(["STATION","DATE_TIME"])[["ENTRIES"]].sum()
station_df["FOUR_HOURS_STATION_ENTRIES"]=station_df.groupby(["STATION"]).ENTRIES.diff()

station_df["FOUR_HOURS_STATION_ENTRIES"]=(
np.where((
    (station_df.ENTRIES ==0) & (station_df.FOUR_HOURS_STATION_ENTRIES < 0)),0 # IN CASE OF COUNTER RESET RET0 
    , 
    np.where((station_df.ENTRIES !=0) & (station_df.FOUR_HOURS_STATION_ENTRIES < 0)
             , abs(station_df.FOUR_HOURS_STATION_ENTRIES), # IN CASE OF COUNTER REVERSE RETURN ABS(DAILY_ENTRIES)
         station_df.FOUR_HOURS_STATION_ENTRIES) # IN CASE NORMAL COUNTER RETURN SAME VALUE AS DAILY_ENTRIES
))


In [ ]:
#drop rows having FOUR_HOURS_STATION_ENTRIES=Nan because it is the start of the chosen period
station_df.dropna(inplace=True)
station_df.reset_index(inplace=True)
station_df["day_of_week"]=station_df["DATE_TIME"].dt.weekday
station_df["hour"]=station_df["DATE_TIME"].dt.hour
station_df["DATE"]=station_df["DATE_TIME"].dt.date

In [ ]:
stations_daily=station_df.groupby(['STATION','DATE'])['FOUR_HOURS_STATION_ENTRIES'].sum().reset_index()

In [ ]:
stations_daily["DATE"] = pd.to_datetime(stations_daily.DATE)
stations_daily["day_of_week"]=stations_daily["DATE"].dt.weekday
stations_daily.sort_values(by='FOUR_HOURS_STATION_ENTRIES',ascending=False).head(10)

## Station with highest average entries per day & Station with lowest average entries per day

In [ ]:
temp_df=stations_daily.groupby(['STATION']).FOUR_HOURS_STATION_ENTRIES.mean().reset_index()
temp_df.sort_values(by='FOUR_HOURS_STATION_ENTRIES',ascending=False).head(10)

## What is the most crowded day for the stations?

In [ ]:
stations_average_of_weekday=station_df.groupby(['STATION','day_of_week']).FOUR_HOURS_STATION_ENTRIES.mean()
stations_average_of_weekday=stations_average_of_weekday.reset_index()

stations_average_of_weekday.sort_values(by='FOUR_HOURS_STATION_ENTRIES',ascending=False)
d={0:"Monday",1:"Tuesday",2:"Wednesday", 3:'Thursday',4:"Friday",5:"Saturday",6:"Sunday"}

stations_average_of_weekday.day_of_week=stations_average_of_weekday.day_of_week.map(lambda x:d[x])

In [ ]:
stations_average_of_weekday.rename(columns={'FOUR_HOURS_STATION_ENTRIES':'AVERAGE_ENTRIES'},inplace=True)

In [ ]:
stations_average_of_weekday.sort_values(by='AVERAGE_ENTRIES',ascending=False).head(10)

## Where are the most crowds on Sunday?

In [ ]:
stations_on_Sunday=stations_average_of_weekday[stations_average_of_weekday['day_of_week']=='Sunday']
stations_on_Sunday.sort_values(by='AVERAGE_ENTRIES',ascending=False).head(10)

## On Monday?

In [ ]:
stations_on_Monday=stations_average_of_weekday[stations_average_of_weekday['day_of_week']=='Monday']
stations_on_Monday.sort_values(by='AVERAGE_ENTRIES',ascending=False).head(10)

## On Tuesday?

In [ ]:
stations_on_Tuesday=stations_average_of_weekday[stations_average_of_weekday['day_of_week']=='Tuesday']
stations_on_Tuesday.sort_values(by='AVERAGE_ENTRIES',ascending=False).head(10)

## On Wednesday?

In [ ]:
stations_on_Wednesday=stations_average_of_weekday[stations_average_of_weekday['day_of_week']=='Wednesday']
stations_on_Wednesday.sort_values(by='AVERAGE_ENTRIES',ascending=False).head(10)

## On Thursady?

In [ ]:
stations_on_Thursday=stations_average_of_weekday[stations_average_of_weekday['day_of_week']=='Thursday']
stations_on_Thursday.sort_values(by='AVERAGE_ENTRIES',ascending=False).head(10)

## On Friday?

In [ ]:
stations_on_Friday=stations_average_of_weekday[stations_average_of_weekday['day_of_week']=='Friday']
stations_on_Friday.sort_values(by='AVERAGE_ENTRIES',ascending=False).head(10)

## On Saturday?

In [ ]:
stations_on_Saturday=stations_average_of_weekday[stations_average_of_weekday['day_of_week']=='Saturday']
stations_on_Saturday.sort_values(by='AVERAGE_ENTRIES',ascending=False).head(10)

## What is the peak day of week for each station?

In [ ]:
temp_df=stations_average_of_weekday.groupby(['STATION'])[['AVERAGE_ENTRIES']].max().reset_index()
temp_df.sort_values(by='AVERAGE_ENTRIES',ascending=False).head(10)

In [ ]:
merged_df=pd.merge(temp_df,stations_average_of_weekday,on=['STATION','AVERAGE_ENTRIES'])
merged_df.sort_values(by='AVERAGE_ENTRIES',ascending=True).head(10)

## Plot the peak day for the station and the number of entries in that peak day

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")
ax = sns.barplot(x=merged_df["day_of_week"], y=merged_df["AVERAGE_ENTRIES"], data=tips)